In [ ]:
# Check input data
!ls '../input/review-lapak-sentiment/'

## Import Package

In [ ]:
import pandas as pd
import numpy as np

## Input Data

In [ ]:
raw_data = pd.read_csv('../input/review-lapak-sentiment/train.csv')
raw_data.head(10)

In [ ]:
# Count Labels
raw_data['label'].value_counts()

## Preprocessing

In [ ]:
# First make a function to delete repetitive alphabet
import itertools

def remove_repeating_characters(text):
    return ''.join(''.join(s)[:1] for _, s in itertools.groupby(text))

# Check our function
remove_repeating_characters('oooofel')

In [ ]:
# Second make a function to remove non alphanumeric
import re

def remove_nonalphanumeric(text):
    text = re.sub('[^0-9a-zA-Z]+', ' ', text)
    return text

# Check our function
remove_nonalphanumeric('o,,,f!!e;;l')

In [ ]:
# Last make a function to convert string to lower case

def to_lower_case(text):
    return text.lower()

# Check our function
to_lower_case('OFEL')

In [ ]:
# Make function that combine them all

def preprocessing_text(text):
    text = remove_repeating_characters(text)
    text = remove_nonalphanumeric(text)
    text = to_lower_case(text)
    
    return text

# Check our function
preprocessing_text('Bagus\n\n\nNamun Akan Lebih Baik Apabila Lebih')

In [ ]:
# Apply function to column 'review_sangat_singkat'

raw_data['review_sangat_singkat'] = raw_data['review_sangat_singkat'].apply(lambda x: preprocessing_text(x))
raw_data.head()

## Extra Preprocessing

In [ ]:
# Make a vector to contain all unique word in 'review sangat singkat'

unique_string = set()
for x in raw_data['review_sangat_singkat']:
    for y in x.split():
        unique_string.add(y)
        
len(unique_string)

In [ ]:
# Count statistics of number of word in review

len_data = [len(x.split()) for x in raw_data['review_sangat_singkat']]
print(np.mean(len_data))
print(np.median(len_data))
print(np.std(len_data))
print(np.min(len_data))
print(np.max(len_data))
print(np.percentile(len_data, 98))

In [ ]:
embed_size = 100 # how big is each word vector
max_features = 23000 # how many unique words to use
maxlen = 20 # max number of words in a comment to use

We will use Tokenizer, check this out at: https://keras.io/preprocessing/text/ 

In [ ]:
# Example
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words = 4)
tokenizer.fit_on_texts(["ini sebuah kalimat hehehe"])
examples = tokenizer.texts_to_sequences(["ini contoh kalimat juga"])
print(examples[0])


In [ ]:
# Real one

tokenizer = Tokenizer(num_words = max_features)
tokenizer.fit_on_texts(raw_data['review_sangat_singkat'])
list_tokenized_train = tokenizer.texts_to_sequences(raw_data['review_sangat_singkat'].values)


In [ ]:
list_tokenized_train[0]

We will use pad sequneces, check this out at: https://keras.io/preprocessing/sequence/

In [ ]:
# Example

from keras.preprocessing.sequence import pad_sequences
pad_sequences(examples, maxlen = maxlen)

In [ ]:
# Real one

X_t = pad_sequences(list_tokenized_train, maxlen= maxlen )

In [ ]:
X_t[0]

## Feature Engineering

We will use word embedding, check this out at: https://www.kaggle.com/ilhamfp31/word2vec-100-indonesian

In [ ]:
import gensim
DIR_DATA_MISC = "../input/word2vec-100-indonesian"
path = '{}/idwiki_word2vec_100.model'.format(DIR_DATA_MISC)
id_w2v = gensim.models.word2vec.Word2Vec.load(path)
print(id_w2v.most_similar('itb'))

In [ ]:
index2word_set = set(id_w2v.wv.index2word)

In [ ]:
word_index = tokenizer.word_index
nb_words = max_features
embedding_matrix = np.zeros((nb_words, embed_size), dtype=np.float32)
unknown_vector = np.zeros((embed_size,), dtype=np.float32) - 1.
for word, i in word_index.items():
    cur = word
    if cur in index2word_set:
        embedding_matrix[i] = id_w2v[cur]
        continue
        
    embedding_matrix[i] = unknown_vector

## Model

In [ ]:
# Import needed packages
# And make needed function


from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GRU
from keras.layers import Bidirectional, GlobalMaxPool1D, SpatialDropout1D, GlobalMaxPooling1D, Concatenate
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras import callbacks

from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall
    
    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def get_model():
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
    x = SpatialDropout1D(0.3)(x)
    x1 = Bidirectional(LSTM(32, return_sequences=True))(x)
    x2 = Bidirectional(GRU(32, return_sequences=True))(x1)
    max_pool1 = GlobalMaxPooling1D()(x1)
    max_pool2 = GlobalMaxPooling1D()(x2)
    conc = Concatenate()([max_pool1, max_pool2])
    x = Dense(1, activation="sigmoid")(conc)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[f1])
    return model

In [ ]:
from sklearn.model_selection import KFold
def get_kfold():
    return KFold(n_splits=5, shuffle=True, random_state=1)

In [ ]:
X = X_t
y = raw_data["label"].values

pred_cv = np.zeros(len(y))
count = 0

for train_index, test_index in get_kfold().split(X, y):
    count += 1
    print(count, end='')
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    es = callbacks.EarlyStopping(monitor='val_f1', min_delta=0.0001, patience=8,
                                             verbose=1, mode='max', baseline=None, restore_best_weights=True)

    rlr = callbacks.ReduceLROnPlateau(monitor='val_f1', factor=0.5,
                                      patience=3, min_lr=1e-6, mode='max', verbose=1)
    
    
    model = get_model()
    model.fit(X_train, 
             y_train, batch_size=16, epochs=4,
             validation_data=(X_test, y_test),
             callbacks=[es, rlr],
             verbose=1)
    
    pred_cv[[test_index]] += model.predict(X_test)[:,0]

In [ ]:
model.save('UseeTV Go Tweets Texts Sentiment Analysis Model.h5')

## Preprocessing UseeTV Go Tweets Text Dataset

In [ ]:
useetv_twitter_data = pd.read_csv( "../input/useetv-go-twitter-tweets-texts-stemmed-dataset/Text_Preprocessing.csv")

useetv_twitter_data.head()

In [ ]:
print("Dataset Tweets Tweets dari Tanggal {} sampai Tanggal {}".format( useetv_twitter_data["date"].min() , useetv_twitter_data["date"].max() ))
print("Dataset Tweets Tweets User dari {} User".format( len( useetv_twitter_data["user_id"].unique() ) ))

In [ ]:
#Lihat Distribusi Bahasa Twieets Dataset Tweets

import seaborn as sns

#sns.set_theme()

useetv_twitter_data["language"].value_counts().plot( kind = "bar" , figsize = ( 20 , 13 ))

In [ ]:
useetv_twitter_data["date"] = pd.to_datetime(useetv_twitter_data["date"], infer_datetime_format=True)

In [ ]:
useetv_twitter_data.dtypes

In [ ]:
useetv_twitter_data["tweet"]

In [ ]:
useetv_twitter_data = useetv_twitter_data.set_index( "created_at" )

In [ ]:
useetv_twitter_data["date"].resample("M").count().plot(figsize=(12,5), grid=True, title="Total Number of Tweets by Date").set(xlabel="Date", ylabel="Total Number of Tweets")

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
import re, string, unicodedata  #modul regular expression
import nltk
from nltk import word_tokenize, sent_tokenize  #Paket ini membagi teks input menjadi kata-kata.,                                  
from nltk.corpus import stopwords

In [ ]:
#preprocessing

New_word_stop_list = ['kak' , 'admin' , 'useetvgo' , 'go' , 'tv' , 'u' , 'see' , 
'yg', 'ya', 'dg', 'dgn', 'nich', 'nih', 'sih', 'si', 'nya', 
              	'lg', 'dulu', 'jgn', 'kl', 'klu', 'klo', 'kalo', 'nge',
                   	'sip', 'spt', 'hallo', 'halo', 'ny', 'd', 'biar', 'skrg',
                   	'bikin', 'bilang', 'tau', 'utk', 'jd', 'yah', 'loh', 'lho', 'aj', 'aja',
                   	'cm', 'banget', 'deh', 'dimana', 'for', 'i', 'coba', 'it', 'sy', 'lumayan',
                   	'kasih', 'tingkat', 'apk', 'usee', 'useetv', 'channel'
                     "min" , "winda" , "youtube" , "spotify" , "dana" ]


def removeStopword(str):
    stop_words = set(stopwords.words('indonesian'))
    word_tokens = word_tokenize(str)
    filtered_sentence = [w for w in word_tokens if not w in stop_words and w not in New_word_stop_list ]
    return ' '.join(filtered_sentence)
#remove sentence which contains only one word
def removeSentence(str): 
    word = str.split()
    wordCount = len(word)
    if(wordCount<=1):
        str = ''
    
    return str
def cleaning(str):
    #remove non-ascii
    str = unicodedata.normalize('NFKD', str).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    #remove URLs
    str = re.sub(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))', '', str)
    #remove punctuations
    str = re.sub(r'[^\w]|_',' ',str)
    #remove digit from string
    str = re.sub("\S*\d\S*", "", str).strip()
    #remove digit or numbers
    str = re.sub(r"\b\d+\b", " ", str)
    #to lowercase
    str = str.lower()
    #Remove additional white spaces
    str = re.sub('[\s]+', ' ', str)
       
    return str
def preprocessing( tweet):

    tweet = str( tweet )
    tweet = removeSentence( tweet )
    tweet = cleaning( tweet )
    tweet  = removeStopword( tweet )

    #print( str )
    
    return tweet

In [ ]:
#test the code
sentences = ["dimana lokasi kuliner jogja yang murah","alamat gudeg yu djum dimana sih, yang enak","s"]
for st in sentences:
    r = preprocessing(st)
    print(r)

In [ ]:
# Preprocessing Data

useetv_twitter_data["preprocessed_tweets_text"] = useetv_twitter_data["tweet"].apply( lambda row : preprocessing( row ) )

useetv_twitter_data.head()

In [ ]:
# Normalisasi Data

normalized_word = pd.read_csv("../input/normalisasi-indonesian-natural-language-processing/normalisasi.csv")

normalized_word_dict = {}

for index, row in normalized_word.iterrows():
    if row[0] not in normalized_word_dict:
        normalized_word_dict[row[0]] = row[1] 

def normalized_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

def normalized_tweets( tweets_text ):

    tweets_words_list = tweets_text.split( " " )

    normalized_tweets_text = ""

    for word in tweets_words_list :

        if word in normalized_word_dict :
          normalized_tweets_text = normalized_tweets_text + " " + normalized_word_dict[ word ]
        else :
          normalized_tweets_text = normalized_tweets_text + " " + word
    
    return normalized_tweets_text


useetv_twitter_data["normalized_and_preprocessed_tweets_text"] = useetv_twitter_data["preprocessed_tweets_text"].apply( lambda row : normalized_tweets( row ) )

useetv_twitter_data.head()

In [ ]:
export = useetv_twitter_data.to_csv( "Normalized_And_Preprocess_Tweets_Text.csv" )

In [ ]:
useetv_twitter_data_after_june_2020 = useetv_twitter_data[ useetv_twitter_data["date"] >= "2020-06-01"]

useetv_twitter_data_after_june_2020.head()

In [ ]:
useetv_twitter_data_after_june_2020 = useetv_twitter_data_after_june_2020.set_index( "created_at")

In [ ]:
useetv_twitter_data_after_june_2020["username"].value_counts()

In [ ]:
Not_From_User_List = ['indihomemaster', 'indihomecs1', 'indihomecare', 'indihome',
       'indihomeborneo', 'indihomekepri', 'indihomebdg70',
       'info_indihome', 'indihomedigital', 'smgindihome',
       'indihome_jakbar', 'bandarindihome', 'indihome_sbs',
       'indihome_ind', 'indihome_diy', 'telkomsel', 'telkomindonesia', 'telkomjabar', 'telkomkti',
       'telkomcare', 'telkompromo' , "useetvcom" , "indihome" , "telkomsel" ]

def get_from_user_or_not( tweets_username ):

    if tweets_username in Not_From_User_List :
        return "False"
    else :
        return "True"

useetv_twitter_data_after_june_2020["from_user_or_not"] = useetv_twitter_data_after_june_2020["username"].apply( lambda row : get_from_user_or_not( row ) )

useetv_twitter_data_after_june_2020_from_user = useetv_twitter_data_after_june_2020[ useetv_twitter_data_after_june_2020["from_user_or_not"] == "True" ]

useetv_twitter_data_after_june_2020_from_user.head()

In [ ]:
useetv_twitter_data_after_june_2020_not_from_user = useetv_twitter_data_after_june_2020[ useetv_twitter_data_after_june_2020["from_user_or_not"] == "False"]

In [ ]:
useetv_twitter_data_after_june_2020_from_user.describe()

In [ ]:
useetv_twitter_data_after_june_2020_not_from_user.describe()

In [ ]:
print("Dataset Tweets Tweets dari Tanggal {} sampai Tanggal {}".format( useetv_twitter_data_after_june_2020_from_user["date"].min() , useetv_twitter_data_after_june_2020_from_user["date"].max() ))
print("Dataset Tweets Tweets User dari {} User".format( len( useetv_twitter_data_after_june_2020_from_user["user_id"].unique() ) ))

In [ ]:
useetv_twitter_data_after_june_2020["date"].resample("M").count().plot( figsize = ( 12, 5 ) , grid= True , title = "Total Number of Tweets by Date").set( xlabel = "Date" , ylabel = "Total Number of Tweets")
useetv_twitter_data_after_june_2020_not_from_user["date"].resample("M").count().plot( figsize = ( 12, 5 ) , grid = True , title = "Total Number of Tweets by Date").set( xlabel = "Date" , ylabel = "Total Number of Tweets" )
useetv_twitter_data_after_june_2020_from_user["date"].resample("M").count().plot(figsize=(12,5), grid=True, title="Total Number of Tweets by Date").set(xlabel="Date", ylabel="Total Number of Tweets")

In [ ]:
# Real one

#tokenizer = Tokenizer(num_words = max_features)
#tokenizer.fit_on_texts( useetv_twitter_data_after_june_2020_from_user["normalized_and_preprocessed_tweets_text"] )
list_tokenized_train = tokenizer.texts_to_sequences( useetv_twitter_data_after_june_2020_from_user["normalized_and_preprocessed_tweets_text"].values)

In [ ]:
list_tokenized_train[0]

In [ ]:
# Real one

X_t = pad_sequences(list_tokenized_train, maxlen= maxlen )

In [ ]:
X_t[0]

In [ ]:
sentiment_probability_prediction_vec = model.predict( X_t )

In [ ]:
sentiment_probability_prediction_vec

In [ ]:
def convert_probability_to_sentiment_prediction( sentiment_probability ):
    
    if sentiment_probability >= 0.5 :
        return 1
    else :
        return 0
    
    
sentiment_prediction_vec = []

sentiment_probability_prediction_vec_new = []

for sentiment_probability_prediction in sentiment_probability_prediction_vec :
    
    sentiment_prediction_vec.append( convert_probability_to_sentiment_prediction( sentiment_probability_prediction ))
    
    sentiment_probability_prediction_vec_new.append( sentiment_probability_prediction )
    

sentiment_prediction_vec



In [ ]:
useetv_twitter_tweets_text_sentiment_analysis_data = pd.DataFrame({"Tweets Text" : useetv_twitter_data_after_june_2020_from_user["tweet"] , 
                                                                   "Normalized and Preprocessed Tweets Text" : useetv_twitter_data_after_june_2020_from_user["normalized_and_preprocessed_tweets_text"] , 
                                                                   "Sentiment Probability Prediction" : sentiment_probability_prediction_vec_new , 
                                                                   "Sentiment Prediction" : sentiment_prediction_vec})

useetv_twitter_tweets_text_sentiment_analysis_data.head()

In [ ]:
useetv_twitter_tweets_text_sentiment_analysis_data.head( 30 )

In [ ]:
useetv_twitter_tweets_text_sentiment_analysis_data["Sentiment Prediction"].value_counts()